# Introduction

In this tutorial we will "fit" the drift diffusion model to experimental reaction time data, that is we will learn how to choose the parameters of the model such that it "optimally" matches the data. In general, a model is given by a parameterized function specifying and constraining a family of allowed dependencies between independent and dependent variables. Also, in general, this functional form is not known *a priori*, and one therefore needs to compare several reasonable candidate models and select the most appropriate one given the data.

## Prerequisites
- psychophysics reaction time distributions
- drift diffusion model
- basic math and statistics
- basic Python

## Objectives
- learn about and apply Ordinary Least Squares & Maximum Likelihood regression
- fit drift diffusion model to simulated and experimental reaction-time data
- perform model selection given a set of candidate models

# General methodology
We will start with an example of the most common fitting methodology.

## Ordinary Least Squares (OLS) Regression

We have data pairs $(x_i,y_i)$ that we think are linearly related but we are not sure what the slope or intercept is that best characterizes this relationship. To find this, we fit the data with a linear model

$\hat{y}_i = \beta_1 x_i + \beta_0$

and estimate the best fitting parameters by minimizing the mean squared error (MSE)

$\sum_i(y_i - \hat{y}_i)^2 = \sum_i (y_i-\beta_1 x_i-\beta_0)^2$

For the case of linear regression, there is actually an analytical solution:

$\beta_1 = \frac{cov(x,y)}{var(x)}$

$\beta_0 = \bar{y} - \hat{\beta_1} x$

but we will use a more general optimization library to start familiarizing ourselves with these tools.

### Exercise 1: OLS

1. Generate N = 100 data pairs $(x_i,y_i)$ using a linear model with normally distributed noise $\epsilon$ and your choice of slope $\beta_1$ and intercept $\beta_0$ parameters.
2. Calculate the analytical estimate for the OLS regression
3. Write a function that returns the mean square error (MSE) for any parameter values.
4. Use an optimization library to numerically find the parameters that minimize the MSE and compare these to the true parameters of the generative model
5. Plot the data, as well as the anlytical and numerical OLS estimate

Tip: Use the `minimize` function from the `scipy.optimize` module

## Maximum Likelihood Estimation (MLE)

We can also fit the model to the data using Maximum Likelihood methods. To do this, we take the following generative model for the data:

$\hat{y}_i = \beta_1 x_i + \beta_0 + \epsilon$

where $\epsilon\sim \mathcal{N}(0,\sigma^2)$ is a normally distributed random variable with mean 0 and variance $\sigma^2$ and $y\sim \mathcal{N}(\beta_1 x+\beta_0,\sigma^2)$.

The probability distribution of $y$ given $x$ is then given by: 

\begin{eqnarray}
\\
p(y_i|x_i,\beta_1,\beta_0) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp[-(y_i-\beta_1 x_i-\beta_0)^2/(2\sigma^2)]
\end{eqnarray}

For a pair $(x_i,y_i)$, the log likelihood of observation $y_i$ is 
\begin{eqnarray}
\log p(y_i|x_i,a,b)
= \log [\frac{1}{\sqrt{2\pi\sigma^2}}\exp[-(y_i-\beta_1 x_i-\beta_0)^2/(2\sigma^2)]] \\
= \log [\frac{1}{\sqrt{2\pi\sigma^2}}] -(y_i-\beta_1 x_i-\beta_0)^2/(2\sigma^2)
\end{eqnarray}

That is: When $\epsilon$ is normally distributed, maximizing the total log likelihood of the data is equivalent to minimizing the mean squared error.


### Exercise: MLE

1. Generate N = 100 data pairs (y,x) using a linear model with normally distributed noise $\epsilon$ and your choice of slope $\beta_1$ and intercept $\beta_0$ parameters.
2. Write a function that returns the total negative log likelihood for any parameter values.
3. Use an optimization library to numerically find the parameters that minimize the negative log likelihood (or equivalently, maximize the log likelihood of the data given the model).

Tip: Use the `minimize` function from the `scipy.optimize` module.

Hint: You can add a very small number to your likelihood to make sure you are not taking $\log(0)$

### Exercise: Likelihood heat map
1. Plot the likelihood heat map as a function of $\beta_1$ and $\beta_0$

**Expected Output**
(with random seed == 1)
<img src="fig/lin_ll_surface.png">

# Application: Fitting the DDM

Now that we have looked at model fitting for a simple case, we can try to fit the DDM to the monkey reaction time data from last class in order to find the best fitting mean, noise and boundary parameters.

As a first step, we will test our ability to recover the parameters of the model on simulated data for which we set the parameters.

Once we are convinced that we can recover the parameters on simulated data, we will fit the model to the experimental monkey data.


## Excercise: Histogram of simulated RTs
1. Generate 5000 trials of simulated RT data using the constant bound DDM function ($\mu = 0.0015, \sigma = 0.05, B = 1$)
2. Plot the simulated RT data using your plot function from yesterday

Tip: We will import a data simulation function `sim_DDM_constant`, a plotting function `plot_rt_distribution` and a function that computes the analytic DDM and returns the (RG) `analytic_ddm` from the module ddm. You can use this module or you can use your own based on the work from the last tutorial.

*Expected output*
<img src="fig/RT_simulated_cb.png">

## Exercise: Likelihood from analytic DDM

1. Implement the following function
```
def get_nll_ddm(parameter, sigma, rts, corrects):
    '''
    Determines the negative loglikelihood of the analytical DDM
    
    Parameters
    ----------
    parameter : array_like of float
        length 2: 1st entry is mu (drift rate), 2nd is B (boundary)
        Note: we pack mu and B in one parameter because we want to
        make it compatible for later use with sp.optimize.minimize
    sigma : float
        DDM standard deviation
    rts : array_like of floats
        reaction times for which the likelihood will be evaluated
    corrects: array_like of bools, same length as rts
        indicates for each rt if it was a correct trial
        
    Returns
    -------
    nll : float
        negative log-likelihood
    '''
```

1. Use the `analytic_ddm` function from the module `ddm` to calculate the log likelihood for a correct trial where RT is $500ms$, $\mu=0.0015$ and $B=1$.
2. What's the log-likelihood for an incorrect trial with otherwise identical parameters?

4. What's the analytical log-likelihood of the decision-variable trajectory from the previous exercise?

## Exercise: Fit DDM to simulated monkey reaction time distribution

Once you are able to evaluate your likelihood function at various parameter values, it's time to fit the simulated data. The goal here is to pass the negative log likelihood to an optimizer that will find the parameters to minimize the total negative log likelihood.

Note that optimizers tend to work better when parameters have the same order of magnitude. Also, the optimization function that we are going to use, `scipy.optimize.minimize`, requires that all parameters that are optimized over are packed into a vector and that this vector is the first argument of the objective function.

- Write a wrapper function that's exactly like `get_nll_ddm`, except that it takes as first argument the vector $(1000 \mu,B)$.

Remember that this will mean rescaling the parameters returned by the optimizer in future exercises!

2. Use the optimizer `minimize` on your negative log likelihood function to maximize the log likelihood of the simulated data. Again $\sigma$ will be fixed at 0.05. 
3. Is the optimization succesful? If yes, you should see "message: 'Optimization terminated successfully.'" in the output. If not, consider using a bounded optimization (check out the bounds input to the function and use method 'SLSQP'). $mu$ and $B$ should be positive.
4. Compare the simulated data with the fitted distribution. To do so, use the analytic_ddm function with the fitted parameter value.

*Expected output*
<img src="fig/RT_sim_fits.png">

## Exercise: Fit DDM to experimental monkey RT data
Now we use the same approach to fit the monkey data for the coherence 0.064.

1. Plot the distribution of the monkey data and compare it with the distribution of the simulated data. Do you notice any difference?
2. Fit the monkey data for coherence 0.064 using the same approach as before. How is the fit?

*Expected output*
<img src="fig/RT_simulated_cb_bins.png">
<img src="fig/RT_monkey_data.png">

## Exercise: DDM with non-decision time
We can potentially improve the fit by having a non-decision time, which is a constant added to all reaction time.

1. Calculate the negative loglikelihood of the monkey data with a non-decision time $t_{nd}$.
2. Maximize the log likelihood with a model of three parameters $\mu,B,t_{nd}$.

# Model Comparison
We will use the $BIC$ (Bayesian Information Criterion) to compare models:
\begin{eqnarray}
-2 \log p(M|y) \approx -2\ln(L) + k\ln(n) \equiv BIC
\end{eqnarray}
where 
- $M$ is the model under consideration, 
- $L$ the likelihood for model $M$,
- $y$ the observed data, 
- $k$ the number of free parameters, 
- $n$ the number of data points (observations)

and the approximation holds for large $n$.

The $BIC$ penalizes more complex models with more parameters. Specifically, in our context, the BIC penalizes the non-decision time model for its extra parameter.

Note that a lower BIC is better and in general a difference of BIC 10 or more is good evidence for the model with the lower BIC.

***Reference***

Wit et al. ‘All models are wrong...’: an introduction to model uncertainty. Statistica Neederlandia (2012).

## Exercise: BIC

1. Compare the BIC of the two models

2. Which model has the smaller BIC (the smaller the better)?

# [Optional] Exercise: Generalizing ML fitting

Above we derived for a linear model function $y$ that when the measurment error $\epsilon$ is normally distributed, then maximizing the total log likelihood of the data is equivalent to minimizing the mean squared error.

- Now assume your data $\{{x_i}, {y_i}\}$ can be described by an arbitrary model function $f(x, \theta$) where $\theta$ represents the function's parameters that you want to recover. Moreover, assume that you know that the measurement errors $\epsilon_i = y_i - f(x_i, \theta)$ are not normally distributed but follow an arbitrary probability distribution $p$. Derive the likelihood of $\theta$ given the data under these assumptions.
- Pick a function (e.g. a difference between two exponentials, or a polynomial multiplied by an exponential) and a probability distribution $p$ and fit the RT data. Compare.